# Superstore Sales ETL Pipeline

This Jupyter notebook implements a full ETL (Extract, Transform, Load) pipeline for the Kaggle Superstore sales dataset. It downloads, loads, cleans, and summarizes the sales data, with logging and error handling.

In [1]:
# Install KaggleHub if needed
# %pip install kagglehub

In [2]:
import kagglehub
import pandas as pd
import os
import logging
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [3]:
# Download dataset from Kaggle
logger.info('Downloading dataset...')
path = kagglehub.dataset_download('dataobsession/superstore-sales-the-data-quality-challenge')
print(f'Kaggle dataset directory: {path}')

100%|██████████| 1.44M/1.44M [00:00<00:00, 70.8MB/s]

Extracting files...
Kaggle dataset directory: /root/.cache/kagglehub/datasets/dataobsession/superstore-sales-the-data-quality-challenge/versions/1


In [4]:
# Find and load CSV
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
if len(csv_files) == 0:
    logger.error('No CSV files found in directory!')
    raise FileNotFoundError('No CSV files found in the dataset directory!')
csv_path = os.path.join(path, csv_files[0])
df = pd.read_csv(csv_path)
print(f'Loaded CSV: {df.shape[0]} rows, {df.shape[1]} columns')
df.head()

Loaded CSV: 12617 rows, 19 columns


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,12618,CA-2017-212321,11/16/2016,11/20/2016,Second Class,BG-11740,United States,Auburn,Washington,98002,West,OFF-AR-10001897,OfficeSupply,Art,Model L Table or Wall-Mount Pencil Sharpener,0.0100,3,NaN,-0.0033
1,12619,CA-2017-128744,01/08/2014,01/11/2014,Same Day,SG-20080,United States,Detroit,Michigan,48234,Central,TEC-CO-10004115,Tech,Copiers,Sharp AL-1530CS Digital Copier,497.0913,14,0.0,28.5634
2,12620,CA-2014-184096,10/06/2016,10/12/2016,First Class,AS-10045,United States,Gaithersburg,Maryland,20877,East,OFF-BI-10003684,OfficeSupply,Binders,Wilson Jones Legal Size Ring Binders,234.6967,14,0.1,NaN
3,12621,US-2014-626204,07/18/2015,07/24/2015,Second Class,DD-13570,United States,La Mesa,California,91941,--,OFF-PA-10003543,OfficeSupply,Paper,Xerox 1985,173.6489,14,0.3,24.5382
4,12622,US-2014-808197,12/25/2016,12/26/2016,Second Class,BD-11605,United States,Modesto,California,95351,West,TEC-PH-10001468,technologies,Phones,Panasonic Business Telephones KX-T7736,1212.5338,11,0.8,-629.6133


In [5]:

def transform_data(df):
    """
    Transform and clean Superstore sales data
    - Removes duplicates
    - Handles missing/invalid dates
    - Fills missing categories
    - Drops records with missing critical fields
    - Calculates total_amount (Quantity × Sales)
    - Adds processing timestamp
    """
    logger.info("Starting data transformation")
    initial_count = len(df)
    df_clean = df.drop_duplicates()
    logger.info(f"Removed {initial_count - len(df_clean)} duplicate records")
    df_clean = df_clean[df_clean['Order Date'] != '']
    df_clean = df_clean.dropna(subset=['Order Date'])
    df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')
    df_clean['Ship Date'] = pd.to_datetime(df_clean['Ship Date'], errors='coerce')
    df_clean['Category'] = df_clean['Category'].fillna('Unknown')
    df_clean['Sub-Category'] = df_clean['Sub-Category'].fillna('Unknown')
    df_clean = df_clean.dropna(subset=['Quantity', 'Sales', 'Profit'])
    df_clean['total_amount'] = df_clean['Quantity'] * df_clean['Sales']
    df_clean['processed_at'] = datetime.now()
    logger.info(f"Transformation complete: {len(df_clean)} records")
    return df_clean


In [6]:
# Run ETL transformation
df_clean = transform_data(df)
print(f'Original records: {len(df)}')
print(f'Cleaned records: {len(df_clean)}')
print('Preview:')
df_clean.head()
df_clean.dtypes

Original records: 12617
Cleaned records: 11737
Preview:


,0
Row ID,int64
Order ID,object
Order Date,datetime64[ns]
Ship Date,datetime64[ns]
Ship Mode,object
Customer ID,object
Country,object
City,object
State,object
Postal Code,int64


Load SQLite database (.db) File

In [9]:
import sqlite3

# Connect to a new (or existing) SQLite database file
conn = sqlite3.connect('superstore_sales_clean.db')

# Write the DataFrame (this creates/replaces the table SUPERSTORE_SALES_CLEAN)
df_clean.to_sql('SUPERSTORE_SALES_CLEAN', conn, if_exists='replace', index=False)

print("Data loaded to SQLite database!")


Data loaded to SQLite database!


In [10]:
# Query the database to verify the number of rows
import pandas as pd

result = pd.read_sql_query("SELECT COUNT(*) as row_count FROM SUPERSTORE_SALES_CLEAN", conn)
print("Number of records loaded:", result['row_count'][0])


Number of records loaded: 11737


In [12]:
# Query the entire database
query = """
SELECT *
FROM SUPERSTORE_SALES_CLEAN;
"""

all_data_df = pd.read_sql_query(query, conn)
display(all_data_df)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country,City,State,Postal Code,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,total_amount,processed_at
0,12618,CA-2017-212321,2016-11-16 00:00:00,2016-11-20 00:00:00,Second Class,BG-11740,United States,Auburn,Washington,98002,...,OFF-AR-10001897,OfficeSupply,Art,Model L Table or Wall-Mount Pencil Sharpener,0.010000,3,NaN,-0.0033,0.030000,2025-11-02 20:53:11.125402
1,12619,CA-2017-128744,2014-01-08 00:00:00,2014-01-11 00:00:00,Same Day,SG-20080,United States,Detroit,Michigan,48234,...,TEC-CO-10004115,Tech,Copiers,Sharp AL-1530CS Digital Copier,497.091300,14,0.00,28.5634,6959.278200,2025-11-02 20:53:11.125402
2,12621,US-2014-626204,2015-07-18 00:00:00,2015-07-24 00:00:00,Second Class,DD-13570,United States,La Mesa,California,91941,...,OFF-PA-10003543,OfficeSupply,Paper,Xerox 1985,173.648900,14,0.30,24.5382,2431.084600,2025-11-02 20:53:11.125402
3,12622,US-2014-808197,2016-12-25 00:00:00,2016-12-26 00:00:00,Second Class,BD-11605,United States,Modesto,California,95351,...,TEC-PH-10001468,technologies,Phones,Panasonic Business Telephones KX-T7736,1212.533800,11,0.80,-629.6133,13337.871800,2025-11-02 20:53:11.125402
4,12623,US-2016-469032,2017-09-08 00:00:00,2017-09-12 00:00:00,Second Class,LB-16795,United States,Covington,Washington,98042,...,TEC-PH-10001795,Tech,Phones,ClearOne CHATAttach 160 - speaker phone,0.010000,3,0.70,-0.0031,0.030000,2025-11-02 20:53:11.125402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11732,25229,US-2015-500157,2015-03-01 00:00:00,2015-03-05 00:00:00,Same Day,NP-18325,United States,Carlsbad,New Mexico,88220,...,OFF-PA-10003228,OfficeSupply,Paper,Xerox 1917,0.010219,8,0.60,-0.0016695038590751308,0.081749,2025-11-02 20:53:11.125402
11733,25230,CA-2016-162355,2016-06-30 00:00:00,2016-07-02 00:00:00,Second Class,PF-19165,United States,Sandy Springs,Georgia,30328,...,FUR-BO-10004695,Furni,Bookcases,O'Sullivan 2-Door Barrister Bookcase in Odessa...,1212.908616,7,0.00,278.21,8490.360313,2025-11-02 20:53:11.125402
11734,25231,CA-2017-672376,2014-08-13 00:00:00,2014-08-13 00:00:00,Standard Class,DH-13075,United States,Utica,New York,13501,...,TEC-AC-10003590,Tech,Accessories,"TRENDnet 56K USB 2.0 Phone, Internet and Fax M...",0.010094,13,0.45,-0.0017448255558996605,0.131226,2025-11-02 20:53:11.125402
11735,25232,CA-2015-165125,2015-05-07 00:00:00,2015-05-11 00:00:00,First Class,CR-12730,United States,Escondido,California,92025,...,FUR-CH-10004063,Furni,Chairs,Global Deluxe High-Back Manager's Chair,903.297774,12,0.00,359.79296011652394,10839.573290,2025-11-02 20:53:11.125402


In [13]:
# Example: get sales by category
query = """
SELECT CATEGORY, SUM(SALES) AS total_sales
FROM SUPERSTORE_SALES_CLEAN
GROUP BY CATEGORY
ORDER BY total_sales DESC;
"""

summary = pd.read_sql_query(query, conn)
print(summary)

       Category   total_sales
0  OfficeSupply  2.658824e+06
1         Furni  9.274752e+05
2  technologies  4.856502e+05
3          Tech  4.777437e+05
